[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/dev/lectures/04.Creating_Dialog_Systems/seminar/notebooks/05.global_local_transitions.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/script_structire.svg" >
<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/node_sructire.svg" >

In [ ]:
from df_engine.core import Actor, Context

def high_priority_node_transition(flow_label, label):
    def transition(ctx: Context, actor: Actor, *args, **kwargs) -> tuple[str, str, float]:
        return (flow_label, label, 2.0)

    return transition

In [ ]:
# imports
from df_engine.core.keywords import TRANSITIONS, RESPONSE, GLOBAL
import df_engine.conditions as cnd
import df_engine.labels as lbl
import re

script = {
    GLOBAL: {
        TRANSITIONS: {
            ("greeting_flow", "node1", 1.1): cnd.regexp(r"\b(hi|hello)\b", re.I),
            ("music_flow", "node1", 1.1): cnd.regexp(r"talk about music"),
            lbl.to_fallback(0.1): cnd.true(),
            lbl.forward(): cnd.all(
                [cnd.regexp(r"next\b"), cnd.has_last_labels(labels=[("music_flow", i) for i in ["node2", "node3"]])]
            ),
            lbl.repeat(0.2): cnd.all(
                [cnd.regexp(r"repeat", re.I), cnd.negation(cnd.has_last_labels(flow_labels=["global_flow"]))]
            ),
        }
    },
    "global_flow": {
        "start_node": {RESPONSE: ""},  # This is an initial node, it doesn't need an `RESPONSE`
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: "Ooops",
            TRANSITIONS: {lbl.previous(): cnd.regexp(r"previous", re.I)},
        },
    },
    "greeting_flow": {
        "node1": {
            RESPONSE: "Hi, how are you?",  # When the agent goes to node1, we return "Hi, how are you?"
            TRANSITIONS: {"node2": cnd.regexp(r"how are you")},
        },
        "node2": {
            RESPONSE: "Good. What do you want to talk about?",
            TRANSITIONS: {lbl.forward(0.5): cnd.regexp(r"talk about"), lbl.previous(): cnd.regexp(r"previous", re.I)},
        },
        "node3": {RESPONSE: "Sorry, I can not talk about that now.", TRANSITIONS: {lbl.forward(): cnd.regexp(r"bye")}},
        "node4": {RESPONSE: "bye"},
    },
    "music_flow": {
        "node1": {
            RESPONSE: "I love `System of a Down` group, would you like to tell about it? ",
            TRANSITIONS: {lbl.forward(): cnd.regexp(r"yes|yep|ok", re.I)},
        },
        "node2": {RESPONSE: "System of a Downis an Armenian-American heavy metal band formed in in 1994."},
        "node3": {
            RESPONSE: "The band achieved commercial success with the release of five studio albums.",
            TRANSITIONS: {lbl.backward(): cnd.regexp(r"back", re.I)},
        },
        "node4": {
            RESPONSE: "That's all what I know",
            TRANSITIONS: {
                ("greeting_flow", "node4"): cnd.regexp(r"next time", re.I),
                ("greeting_flow", "node2"): cnd.regexp(r"next", re.I),
            },
        },
    },
}


In [ ]:
from df_runner import ScriptRunner

runner = ScriptRunner(
    script,
    start_label=("global_flow", "start_node"),
    fallback_label=("global_flow", "fallback_node"),
)

In [ ]:
runner.start()